In [3]:
from collections import deque
def bfs(startstate):
    frontier =  deque([startstate])
    prev = {startstate:None}
    while frontier:
        state = frontier.popleft()  
        if goaltest(state):
            return prev, state
        for v in nextstates(state):
            if v not in prev:
                frontier.append(v)
                prev[v] = state
    return None, None
                
def dfs(startstate):
    frontier =  [startstate]
    prev = {startstate:None}
    while frontier:
        state = frontier.pop()   
        if goaltest(state):
            return prev,state
        for v in reversed(nextstates(state)):                  # nach rechts runter, für links: reversed(nextstates(state))
            if v not in prev:
                frontier.append(v)
                prev[v] = state
    return None, None

def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]        
    path.reverse()
    return path


### Zahlenbaum
Wir suchen die Zahl 18

In [4]:
def nextstates(state):
    if state in baum:
        return baum[state]
    else: return []

def goaltest(state):
    return state == 13
 
baum = {1:[12,6,7], 12: [14,22,10], 6: [4,8], 7: [2,18], 22:[3,9], 2:[13]}
 
startstate = 1
prev, goalstate = dfs(startstate)
print(reconstructPath(prev,goalstate))
 

[1, 7, 2, 13]


### 8-Puzzle

In [5]:
def swap(state,i,j):
    '''
    state: tuple
    i,j: gueltige Indizes für das tuple
    returns: tuple mit vertauschten Werten an Positionen i und j
    '''
    tmp = list(state)
    tmp[i],tmp[j] = tmp[j],tmp[i]
    return tuple(tmp)

def show(state):
    import math
    '''
    printed state als 3x3 Matrix
    '''
    for i in range(len(state)):
        k = math.sqrt(len(state))
        if i%k==0:
            print()
        print(f'{state[i]:2}',end = ' ')
    print()

def nextstates(state):
    tmp = []
    i = state.index(0)
    for j in tauschDict[i]:
        tmp.append(swap(state,i,j))
    return tmp

def goaltest(state):
    return state == (0,1,2,3,4,5,6,7,8) 

In [6]:
%%time
tauschDict = {0:[1,3],1:[2,4,0],2:[5,1],3:[4,6,0],4:[5,7,3,1],5:[8,4,2],6:[7,3],7:[8,6,4],8:[7,5]}
startstate = (7,2,4,5,0,6,8,3,1)   
#startstate = (8,2,7,3,4,6,5,0,1)
#startstate = (1,2,3,4,5,6,8,7,0)    # nicht lösbar
prev, goalstate = bfs(startstate)
if prev is None:
    print('nicht lösbar')
else:
    path = reconstructPath(prev,goalstate)
    print(f'Lösung in {len(path)-1} Zügen gefunden.')

 

Lösung in 26 Zügen gefunden.
Wall time: 395 ms


Das dictionary *prev* dient mehreren Zwecken:
 - wir merken uns, was wir schon untersucht haben (explored) oder was noch zur Untersuchung vorgesehen ist (frontier)
 - wir merken uns den Elternknoten

In [9]:
# for state in path:
#     show(state)

### 15-Puzzle

In [10]:
tauschDict = {0:[1,4],1:[2,5,0],2:[3,6,1],3:[7,2],4:[5,8,0],5:[6,9,4,1],6:[7,10,5,2],7:[11,6,3],8:[9,12,4]}
tauschDict.update({9:[10,13,8,5],10:[11,14,9,6],11:[15,10,7],12:[13,8],13:[14,12,9],14:[15,13,10],15:[14,11]})

# startstatexx - xx: Anzahl Züge, Zeit bfs, Zeit astar(zip), Zeit iterative deepening
startstate10  = (2,10,3,4,1,0,6,8,5,9,7,11,13,14,15,12)   # 33 ms  
startstate15  = (2,10,4,8,1,6,3,11,5,9,7,0,13,14,15,12)   # 826 ms  
startstate20  = (2,10,4,8,5,1,6,11,0,9,3,12,13,14,7,15)   # 14.5 s  - 3.6s - 27 s
startstate21  = (10,0,4,8,2,1,6,11,5,9,3,12,13,14,7,15)   # 27.1 s  - 19 ms  -  25.9
startstate23  = (10,4,6,8,2,1,0,11,5,9,3,12,13,14,7,15)   # 2.2 min  - 83.5 ms
startstate35 = (5,4,8,11,3,1,14,10,13,7,2,0,9,6,15,12)    # ? - 1:27 min - 17 ms

goalstate = (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0)

def goaltest(state):
    return state == goalstate

In [12]:
%%time
prev, goalstate = bfs(startstate10)
path = reconstructPath(prev,goalstate)
len(path)                  

Wall time: 24 ms


11

### Best first search

Best-first search ist ein allgemeiner Begriff für eine Suche, bei dem der state als nächstes aus der frontier geholt wird,
der nach einer wie immer gearteteten Evaluations-Funktion, den besten (=niedrigsten) Wert hat.

### Greedy

Greedy ist eine Form des best-first search. Der state ist am besten, der die größte Ähnlichkeit (oder den kleinsten Abstand) zum goalstate hat.


In [13]:
from heapq import heappop, heappush
def greedy(s):
    frontier =[(h(s),s)]  
    prev = {s:None}
    while frontier:
        _ ,state = heappop(frontier)  
        if goaltest(state):
            return prev, state
        for v in nextstates(state):
            if v not in prev:
                heappush(frontier,(h(v),v))
                prev[v] = state
    return None, None 

def astar(s):
    frontier =[(h(s),s)]  
    prev = {s:None}
    g = {s:0}                         # backword costs: hier die Anzahl Züge
    while frontier:
        _ ,state = heappop(frontier)  # die Kosten braucht man an der Stelle nicht
        if goaltest(state):
            return prev, state
        for v in nextstates(state):
            gg = g[state] + 1
            if v not in prev or gg < g[v]:
                g[v] = gg
                heappush(frontier,(g[v]+h(v),v))
                prev[v] = state
    return None, None

def h(state):
    ''' returns: anzahl der von der endgültigen position verschiedenen stellen'''
    return sum(a!=b for a,b in zip(goalstate,state))



In [14]:
%%time
prev, goalstate = astar(startstate23)
path = reconstructPath(prev,goalstate)
print(len(path))

24
Wall time: 70 ms


In [15]:
%%time
prev, goalstate = greedy(startstate35)
path = reconstructPath(prev,goalstate)
print(len(path))

102
Wall time: 20 ms


###  Depth Limited Search und Iterative Deepening

In [57]:
def dls(startpath, maxlevel):
    frontier =  [startpath]
    result = "FAILURE"
    while frontier:
        path = frontier.pop()   
        if goaltest(path):
            return path
        for p in nextpaths(path):
            if len(p) > maxlevel: 
                result = "CUTOFF"
            else:
                frontier.append(p)
    return result

def iterativeDeepening(startstate):
    for i in range(20,30,1):
        startpath = [startstate]
        result = dls(startpath,i)
        if result != "CUTOFF":
            return result
        
def nextpaths(path):
    tmp = []
    for state in nextstates(path[-1]):
        if state not in path:
            p = path.copy()
            p.append(state)
            tmp.append(p)
    return tmp

def goaltest(path):
    return path[-1] == goalstate                   

In [62]:
%%time
path = iterativeDeepening(startstate23)
len(path)-1


Wall time: 2min 30s


23